In [1]:
import math               # Libreria que se usa para realizar operaciones matemáticas y funciones
import random             # Libreria usada para generar números aleatorios
import numpy as np        # Encuentra una raiz real entre 0.5 y 1.5
import wave
 # Módulo de onda de importación
import matplotlib.pyplot as plt
 # Se usa para dibujar formas de onda
import os
 # Se utiliza para el procesamiento del sistema, como la lectura de archivos de audio locales.
 

In [2]:
longCrom = 175576
K = 50
M = 100
pm = 0.01

In [3]:
def genera(K,longCrom):
    import math               # Libreria que se usa para realizar operaciones matemáticas y funciones
    import random             # Libreria usada para generar números aleatorios
    import numpy as np        # Encuentra una raiz real entre 0.5 y 1.5
    import wave
    import sounddevice as sd 
    import matplotlib.pyplot as plt


    # Módulo de onda de importación
    
    # Se usa para dibujar formas de onda
    import numpy as np
    # Se utiliza para calcular datos de forma de onda
    import os
    # Se utiliza para el procesamiento del sistema, como la lectura de archivos de audio locales.
    f = wave.open(r"conversacion1.wav",'rb' )
    params = f.getparams ()
    nchannels, sampwidth, framerate, nframes = params [: 4] # Número de 0 ~ 4 antes
    print(nchannels,sampwidth,framerate,nframes)
    # Leer datos de forma de onda
    strData = f.readframes(nframes)
    # Convertir datos de forma de onda en matriz
    waveData = np.fromstring(strData,dtype=np.int16)
    waveData = waveData*1.0/(max(abs(waveData)))
    time = np.arange (0, nframes) * (1.0 / framerate) #Calcular el tiempo de audio
    waveData = np.reshape(waveData,[nframes,nchannels]) 
    f.close()
    #Trabajaremos con  numeros enteros
    for i in range(len(waveData)):       
        waveData[i]=int((waveData[i])*1000)
    sd.play(waveData, blocking=True)
    print(len(waveData))
    return waveData



In [4]:
def genera(K,longCrom):
    import math               # Libreria que se usa para realizar operaciones matemáticas y funciones
    import random             # Libreria usada para generar números aleatorios
    import numpy as np        # Encuentra una raiz real entre 0.5 y 1.5
    import wave
    import sounddevice as sd 
    # Módulo de onda de importación
    import matplotlib.pyplot as plt
    # Se usa para dibujar formas de onda
    import numpy as np
    # Se utiliza para calcular datos de forma de onda
    import os
    # Se utiliza para el procesamiento del sistema, como la lectura de archivos de audio locales.
    f = wave.open(r"conversacion1.wav",'rb' )
    params = f.getparams ()
    nchannels, sampwidth, framerate, nframes = params [: 4] # Número de 0 ~ 4 antes
    #print(nchannels,sampwidth,framerate,nframes)
    # Leer datos de forma de onda
    strData = f.readframes(nframes)
    # Convertir datos de forma de onda en matriz
    waveData = np.fromstring(strData,dtype=np.int16)
    waveData = waveData*1.0/(max(abs(waveData)))
    time = np.arange (0, nframes) * (1.0 / framerate) #Calcular el tiempo de audio
    waveData = np.reshape(waveData,[nframes,nchannels]) 

    #def genera(K,longCrom):
    Pob_nueva = np.zeros([51, longCrom], dtype = float)
    cromosoma = np.zeros([longCrom], dtype = float)
    j = 0
    sd.play(np.array(waveData), blocking=True)
    #creamos los 50 audios
    while j < K:

        for n in range(len(waveData)):
            #le añadimos ruido
            l=random.randint(0, 2)
            if(l==0):
                if(n%2==0 or n%2==5 or n%2==3):
                    cromosoma[n]=random.randint(-500, 1005)
            if(l==1):
                if(n%2==5 ):
                    cromosoma[n]=random.randint(-50, 10)
            if(l==2):
                if(n%2==11 ):
                    cromosoma[n]=random.randint(-500, 1005)
                else:
                    cromosoma[n]=(waveData[n])*1000 
                
            
        Pob_nueva[j] = cromosoma.reshape(-1)

        j +=1
 
    return Pob_nueva


In [5]:
import sounddevice as sd 
def evalua( Pob_nueva):
    Apt_total = 0
    vectorX = np.zeros(K, dtype = float)
    probab = np.zeros(K, dtype = float) 
    aptitud = np.zeros(K, dtype = float) 
    mejor = 50.
    mejorX = 0.
  
    i = 0
    while(i < K):
        #Nosotros somo la funcion de aptitud (segun la calificacion de cada uno obtendremos el final con mas parecido al original)        
        sd.play(np.array(Pob_nueva[i]), blocking=True)
        apt = input("Aptitud de 0 al 10. Audio N° "+str(i)+" de 50")
        aptitud[i] = (10 - abs(float(apt)))*100
        i += 1
    Apt_total = float(sum(aptitud))
    probab = 10 * [j/Apt_total for j in aptitud]
    maxprobab = np.amax(probab)
    maxIndex = np.argmax(probab)
    mejorx = vectorX[maxIndex] 
    probab[maxIndex] = 0.99
    return probab, vectorX 


# las funciones las creamos antes

In [6]:
def cruce(Pob_nueva, Probabilidad): 
    maxprobab = np.amax(prob_cromosoma)
    maxIndex = np.argmax(prob_cromosoma)
    len(Pob_nueva)
    i = 0
    while (i < K-1):
        mejorCrom = Pob_nueva[maxIndex]
        if Probabilidad[i] < 0.97:
            rand = random.randint(2,longCrom-1) # Posición de cruce aleatoria
            padre1 = Pob_nueva[i]
            padre2 = Pob_nueva[i+1]
            j = rand
            while(j < longCrom):
                bit = padre1[j]
                padre1[j] = padre2[j]
                padre2[j] = bit
                j += 1  
        else:
            if maxIndex %2 == 0:
                Pob_nueva[i] = Pob_nueva[maxIndex]
            else:
                Pob_nueva[i+1] = Pob_nueva[maxIndex] 
        i += 2
    return Pob_nueva

In [7]:
def muta(Pob_nueva, pm):
    totalbits = K * longCrom
    segmento = 1/pm
    n_segmentos = totalbits/segmento
    i =0
    while(i < n_segmentos-1):
        aleatorio = random.randint(0, segmento-1)
        posic = int(i * segmento + aleatorio)
        y = int((posic/longCrom))
        cromosoma = Pob_nueva[y]
        x = posic - longCrom * y 
        # La magnitud es de 50 unidades
        if (cromosoma[x-1]-50 <0):
            cromosoma[x-1] = (cromosoma[x-1]-50)*-1
        else:
            cromosoma[x-1] = (cromosoma[x-1]-50)
        i +=1    
    return Pob_nueva

In [8]:
def seleccion_ruleta(poblacion, probabilidad):
    chosen = []
    while len(chosen)<= K:
        for n in range(K):
            r = random.random()
            for (i, individuo) in enumerate(poblacion):
                #print(i)
                if i > K:
                    break
                if r <= probabilidad[i]:
                    chosen.append(list(individuo))
                    break
    return chosen

In [9]:
Pob_nueva3 = genera(K,longCrom) 
prob_cromosoma,vectorX = evalua(Pob_nueva3)
i = 0
while (i < M):
    Pob_vieja = Pob_nueva3
    Pob_nueva1 = seleccion_ruleta(Pob_vieja, prob_cromosoma)
    Pob_nueva2 = cruce(Pob_nueva1, prob_cromosoma)
    Pob_nueva3 = muta(Pob_nueva2, pm)
    prob_cromosoma, vectorX = evalua(Pob_nueva3)
    maxprobab = np.amax(prob_cromosoma)
    maxIndex = np.argmax(prob_cromosoma)
    mejorx = vectorX[maxIndex]
    x = mejorx
    # Escucharemos el nuevo
    sd.play(np.array(mejorx), blocking=True)

    i += 1

/tmp/ipykernel_15417/1673639031.py:21: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  waveData = np.fromstring(strData,dtype=np.int16)


KeyboardInterrupt: Interrupted by user